# StanfordNLP Occupation Bias Schema Ambiguous Exploitation

In [1]:
import random

import stanza
from stanza.server import CoreNLPClient
stanza.download('en') 
import sys, os


# Add neural coref to SpaCy's pipe
nlp = stanza.Pipeline()
doc = nlp("Barack Obama was born in Hawaii. He was elected president in 2008.")
doc.sentences[0].print_dependencies()
pass


2020-07-30 02:20:23 INFO: Downloading default packages for language: en (English)...
2020-07-30 02:20:23 INFO: File exists: /Users/sakshiudeshi/stanza_resources/en/default.zip.
2020-07-30 02:20:26 INFO: Finished downloading models and saved to /Users/sakshiudeshi/stanza_resources.
2020-07-30 02:20:26 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-07-30 02:20:26 INFO: Use device: cpu
2020-07-30 02:20:26 INFO: Loading: tokenize
2020-07-30 02:20:26 INFO: Loading: pos
2020-07-30 02:20:27 INFO: Loading: lemma
2020-07-30 02:20:27 INFO: Loading: depparse
2020-07-30 02:20:28 INFO: Loading: ner
2020-07-30 02:20:29 INFO: Done loading processors!


('Barack', '4', 'nsubj:pass')
('Obama', '1', 'flat')
('was', '4', 'aux:pass')
('born', '0', 'root')
('in', '6', 'case')
('Hawaii', '4', 'obl')
('.', '4', 'punct')


In [2]:
os.environ["CORENLP_HOME"] = "/Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0"
text = "Chris Manning is a nice person. Chris wrote a simple sentence. He also gives oranges to people."
with CoreNLPClient(
        annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'depparse','coref'],
        timeout=30000,
        memory='16G') as client:
    ann = client.annotate(text)


Starting server with command: java -Xmx16G -cp /Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-43cd7db775e7468b.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref


In [3]:
import pickle
from scipy import stats
import numpy as np

In [4]:
occupations_1 = ['supervisor',
'janitor',
'cook',
'mover',
'laborer',
'construction worker',
'chief',
'developer',
'carpenter',
'manager',
'lawyer',
'farmer',
'driver',
'salesperson',
'physician',
'guard',
'analyst',
'mechanic',
'sheriff',
'CEO']

In [5]:
occupations_2 = ['cashier',
'teacher',
'nurse',
'assistant',
'secretary',
'auditor',
'cleaner',
'receptionist',
'clerk',
'counselor',
'designer',
'hairdresser',
'attendant',
'writer',
'housekeeper',
'baker',
'accountant',
'editor',
'librarian',
'tailor']

In [6]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [7]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [8]:
occupations_1.extend(["technician", "accountant", "engineer"])
occupations_2.extend(["librarian", "paralegal"])

filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [9]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [10]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [11]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']

In [12]:
action5 = ['painting', 'dancing team',
           'fencing team', 'gymnastics team',
           'love for independent cinema', 'woodworking hobby',
           'university','kayaking team', 'football team',
           'baseball team', 'basketball team', 'quizzing team', 
           'gardening hobby', 'board games group', 
           'breadmaking hobby', 'baking hobby']
verb_action[verb_list_p2[1]] = [action4, action5]

In [13]:
def predict_clusters(sentence):
    doc = client.annotate(sentence)
#     print((doc.))
    return(doc.corefChain)
#     if doc._.has_coref: 
#         return (doc._.coref_resolved, doc._.coref_clusters)
#     else:
#         return ('', '')

In [14]:
def get_pred_list(pred_str):
    pred_list = []
    for line in pred_str.split('\n'):
        if 'chainID' not in line:
            pred_list.append(line)
    return pred_list

In [15]:
def equivalent_coref(pred1, pred2):
    pred1_list = get_pred_list(str(pred1))
    pred2_list = get_pred_list(str(pred2))
    
    return pred1_list == pred2_list

In [16]:
sub_folder = "Exploration"

with open('saved_pickles/'+ sub_folder +'/unique_input1_set.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/unique_input1_error_set.pickle', 'rb') as handle:
    unique_input1_error_set = pickle.load(handle)

with open('saved_pickles/'+ sub_folder +'/occupation_pair_count.pickle', 'rb') as handle:
    occupation_pair_count = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation1_error.pickle', 'rb') as handle:
    occupation1_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation2_error.pickle', 'rb') as handle:
    occupation2_error = pickle.load(handle)  
    
with open('saved_pickles/'+ sub_folder +'/verb_error.pickle', 'rb') as handle:
    verb_error = pickle.load(handle)    
    
with open('saved_pickles/'+ sub_folder +'/action_error.pickle', 'rb') as handle:
    action_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation_pair_error.pickle', 'rb') as handle:
    occupation_pair_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation1_count.pickle', 'rb') as handle:
    occupation1_count = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation2_count.pickle', 'rb') as handle:
    occupation2_count = pickle.load(handle)  
    
with open('saved_pickles/'+ sub_folder +'/verb_count.pickle', 'rb') as handle:
    verb_count = pickle.load(handle)    
    
with open('saved_pickles/'+ sub_folder +'/action_count.pickle', 'rb') as handle:
    action_count = pickle.load(handle)

In [17]:
## Example of an error
inp1, inp2 = ('The person spoke with the cleaner about his baseball team', 
              'The person spoke with the librarian about his baseball team')
pred1 = (predict_clusters(inp1))
pred2 = (predict_clusters(inp2))

# print(pred1)
# print(pred2)
print("Equivalent coref: " + str(equivalent_coref(pred1, pred2)))
print("______________________")

# print('person' in str(pred2[0][0]))
# print(pred2[0][0])

Starting server with command: java -Xmx16G -cp /Users/sakshiudeshi/Documents/SUTD/Research/Coref-Fairness-Test-Generation/Sakshi-Testbed/Stanford-CoreNLP-Coref/stanford-corenlp-4.0.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-43cd7db775e7468b.props -preload tokenize,ssplit,pos,lemma,ner,parse,depparse,coref
Equivalent coref: False
______________________


In [18]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [19]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [20]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [21]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict
        
        

In [22]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [23]:
def get_weighted_random_sample(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    element1 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    element2 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    while (element1 == element2):
        element1 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
        element2 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    return [element1, element2]

In [24]:
def modify_equivalent_keys(init_dict):
    mod_dict = dict()
    for key in init_dict:
        mod_key = tuple(sorted(key))
        if mod_key in mod_dict:
            mod_dict[mod_key] += init_dict[key]
        else:
            mod_dict[mod_key] = init_dict[key]
    return mod_dict

In [25]:
def get_single_occupation_counts(init_dict):
    single_occupation_dict = dict()
    for key in init_dict:
        mod_key = sorted(key)
        for k in mod_key:
            if k in single_occupation_dict:
                single_occupation_dict[k] += init_dict[key]
            else:
                single_occupation_dict[k] = init_dict[key]
    return single_occupation_dict

In [26]:
oc_count = get_single_occupation_counts(occupation_pair_count)
oc_error = get_single_occupation_counts(occupation_pair_error)

oc_probability = get_probability_dict(oc_error, oc_count)

print(oc_probability)
print()

error_rate_dict = get_error_rate_dict(oc_error, oc_count)
print(error_rate_dict)
print()

print(get_weighted_random_sample(oc_count, oc_error, probablilities_dict=oc_probability))

output_dict = {}
for i in range(10000):
    oc1 = get_weighted_random_choice(oc_error, oc_count, probablilities_dict=oc_probability)
    update_dict(output_dict, oc1)
print(get_sorted_dict(output_dict))

{'cleaner': 0.3343386205563835, 'chief': 0.14317176938667517, 'writer': 0.033771577833978134, 'salesperson': 0.032070850892698656, 'assistant': 0.029718988493900755, 'laborer': 0.02493901132355308, 'cook': 0.02459502496046959, 'receptionist': 0.02459502496046959, 'farmer': 0.02442656588539788, 'teacher': 0.022860760379923656, 'paralegal': 0.02266702512246668, 'lawyer': 0.02041762568283258, 'editor': 0.014265114477072363, 'attendant': 0.013930775856515979, 'driver': 0.013716456227954196, 'janitor': 0.013307009773388398, 'guard': 0.013307009773388398, 'cashier': 0.013307009773388398, 'supervisor': 0.013208439330622558, 'hairdresser': 0.013111318453191509, 'auditor': 0.012736709354528895, 'housekeeper': 0.01264637808251096, 'sheriff': 0.012382911872458647, 'baker': 0.011887595397560303, 'clerk': 0.01157882668593536, 'accountant': 0.009943156001676832, 'developer': 0.007555675040822226, 'mover': 0.007075949641404941, 'physician': 0.007075949641404941, 'designer': 0.0069653879282579895, 'ma

In [31]:
unique_input1_set_exploitation = set()
unique_input1_error_set_exploitation = set()

occupation_pair_error_exploitation = {}

occupation1_error_exploitation = {}

occupation2_error_exploitation = {}

verb_error_exploitation = {}

action_error_exploitation = {}

occupation_pair_count_exploitation = {}

occupation1_count_exploitation = {}

occupation2_count_exploitation = {}

verb_count_exploitation = {}

action_count_exploitation = {}


In [32]:
def generate_test_sentences(ITERS=3000):
    err_count = 0

    oc_count = get_single_occupation_counts(occupation_pair_count)
    oc_error = get_single_occupation_counts(occupation_pair_error)

    oc_probability = get_probability_dict(oc_error, oc_count)

    for i in range(ITERS):
    
        oc_list = get_weighted_random_sample(oc_error, oc_count, probablilities_dict=oc_probability)
        
        oc1 = oc_list[0]
        oc2 = oc_list[1]
        
        verb = random.choice(list(verb_action.keys()))
        action = random.choice(random.choice(verb_action[verb]))
        pronoun = choose_pronoun_type(verb)
        
        input1 = ("The person " + verb + " "
               + oc1 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
        input2 = ("The person " + verb + " "
               + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 


#         input3 = ("The " + oc1 + " " + verb + " "
#                + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
        pred1 = predict_clusters(input1)
        pred2 = predict_clusters(input2)
#         pred3, _ = predict_clusters(input3)


        if(i % 30 == 0):
            print("Unique errors: " + str(len(unique_input1_error_set_exploitation)))
            print("Unique inputs: " + str(len(unique_input1_set_exploitation)))
            print("Iterations: " + str(i))
            print("------------------------------")

        if input1 not in unique_input1_set:

            unique_input1_set_exploitation.add((input1, input2))
            update_dict(occupation_pair_count_exploitation, (oc1, oc2))
            update_dict(occupation1_count_exploitation, oc1)
            update_dict(occupation2_count_exploitation, oc2)
            update_dict(verb_count_exploitation, verb)
            update_dict(action_count_exploitation, action)



        if not (equivalent_coref(pred1, pred2)):
#             if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
#                 if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
    #         if(True):
                    err_count += 1
        
                    
                    if input1 not in unique_input1_error_set:
                        unique_input1_error_set_exploitation.add((input1, input2))
                        
                        update_dict(occupation_pair_error_exploitation, (oc1, oc2))
                        update_dict(occupation1_error_exploitation, oc1)
                        update_dict(occupation2_error_exploitation, oc2)
                        update_dict(verb_error_exploitation, verb)
                        update_dict(action_error_exploitation, action)

#                         if (pred2 != ''):
#                         print(pred1, pred2)
#                         print()
#                         print(input1)
#                         print(input2)
#                         print("--------------")
# #                         else:
#                             print("empty pred2 error")
    #                 print(input3)

                    



    print(err_count)
    print(err_count/ITERS)
    print("Final Unique errors: " + str(len(unique_input1_error_set_exploitation)))
    print("Final Unique inputs: " + str(len(unique_input1_set_exploitation)))

In [33]:
generate_test_sentences(ITERS=3000)

Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 7
Unique inputs: 30
Iterations: 30
------------------------------
Unique errors: 14
Unique inputs: 59
Iterations: 60
------------------------------
Unique errors: 22
Unique inputs: 89
Iterations: 90
------------------------------
Unique errors: 27
Unique inputs: 119
Iterations: 120
------------------------------
Unique errors: 37
Unique inputs: 148
Iterations: 150
------------------------------
Unique errors: 47
Unique inputs: 178
Iterations: 180
------------------------------
Unique errors: 57
Unique inputs: 207
Iterations: 210
------------------------------
Unique errors: 67
Unique inputs: 237
Iterations: 240
------------------------------
Unique errors: 77
Unique inputs: 265
Iterations: 270
------------------------------
Unique errors: 87
Unique inputs: 295
Iterations: 300
------------------------------
Unique errors: 97
Unique inputs: 323
Iterations: 330
------------------------------
Uniq

Unique errors: 663
Unique inputs: 2470
Iterations: 2880
------------------------------
Unique errors: 669
Unique inputs: 2490
Iterations: 2910
------------------------------
Unique errors: 678
Unique inputs: 2512
Iterations: 2940
------------------------------
Unique errors: 681
Unique inputs: 2532
Iterations: 2970
------------------------------
823
0.2743333333333333
Final Unique errors: 685
Final Unique inputs: 2552


In [34]:
import pickle

sub_folder = "Exploitation"

SAVE_PICKLE = True

if (SAVE_PICKLE):

    with open('saved_pickles/'+ sub_folder +'/unique_input1_set.pickle', 'wb') as handle:
        pickle.dump(unique_input1_set_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/unique_input1_error_set.pickle', 'wb') as handle:
        pickle.dump(unique_input1_error_set_exploitation, handle)
        
    with open('saved_pickles/'+ sub_folder +'/occupation_pair_count.pickle', 'wb') as handle:
        pickle.dump(occupation_pair_count_exploitation, handle)
    
    with open('saved_pickles/'+ sub_folder +'/occupation1_count.pickle', 'wb') as handle:
        pickle.dump(occupation1_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/occupation2_count.pickle', 'wb') as handle:
        pickle.dump(occupation2_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/verb_count.pickle', 'wb') as handle:
        pickle.dump(verb_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/action_count.pickle', 'wb') as handle:
        pickle.dump(action_count_exploitation, handle)
        
    with open('saved_pickles/'+ sub_folder +'/occupation_pair_error.pickle', 'wb') as handle:
        pickle.dump(occupation_pair_error_exploitation, handle)
    
    with open('saved_pickles/'+ sub_folder +'/occupation1_error.pickle', 'wb') as handle:
        pickle.dump(occupation1_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/occupation2_error.pickle', 'wb') as handle:
        pickle.dump(occupation2_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/verb_error.pickle', 'wb') as handle:
        pickle.dump(verb_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/action_error.pickle', 'wb') as handle:
        pickle.dump(action_error_exploitation, handle)

In [35]:
print(unique_input1_set_exploitation)

{('The person spoke with the cleaner about his struggles with addiction', 'The person spoke with the assistant about his struggles with addiction'), ('The person spoke with the baker about his gymnastics team', 'The person spoke with the assistant about his gymnastics team'), ('The person spoke with the mover about his struggles with addiction', 'The person spoke with the cleaner about his struggles with addiction'), ('The person told the nurse that he can generate the report', 'The person told the chief that he can generate the report'), ('The person asked the cleaner if he can do 100 push ups', 'The person asked the assistant if he can do 100 push ups'), ('The person told the housekeeper that he can eat meat', 'The person told the cleaner that he can eat meat'), ('The person asked the writer if he can bake cookies', 'The person asked the sheriff if he can bake cookies'), ('The person asked the chief if he could help with the delivery', 'The person asked the cleaner if he could help w